In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import polars as pl
import gc
base_path = '/content/drive/MyDrive/AKI_공유/3.0'
icu_path = os.path.join(base_path, 'icu')
temp_dir = '/content/drive/MyDrive/AKI_공유/3.0/full_join'
pl.Config.set_tbl_rows(1000)# 출력할 행의 수를 설정 (예: 10)
pl.Config.set_tbl_cols(100)# 출력할 열의 수를 설정 (예: 30)
pl.Config.set_tbl_width_chars(1000)


AKI_Label_data_path = os.path.join(temp_dir, '8-procedureevents.parquet')

result = pl.read_parquet(AKI_Label_data_path)
result = result.rename({"value_labevent": "serumCreatinine"})


# filtered_result=result.filter(pl.col('itemid').is_in(emergency_surgical_itemids + non_emergency_surgical_itemids + [50912]) )#| pl.col('itemid').is_in(220615))
filtered_result=result.filter(pl.col('itemid').is_in(50912) )



print("Unique itemids in the dataset:", filtered_result['itemid'].unique())


del result
gc.collect()


Unique itemids in the dataset: shape: (1,)
Series: 'itemid' [i64]
[
	50912
]


3

In [ ]:
##baseline_creatinine 계산하기~~
# charttime을 datetime 형식으로 변환
#filtered_result = filtered_result.with_columns(pl.col("charttime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False))


# subject_id별로 serumCreatinine의 최소값을 계산하여 baseline_creatinine을 추가
baseline_df = (
    filtered_result.group_by('subject_id')
      .agg(pl.col('serumCreatinine').min().alias('baseline_creatinine'))
)

# 원래 데이터프레임에 baseline_creatinine 병합
filtered_result = filtered_result.join(baseline_df, on='subject_id')

# 결과 출력
print(filtered_result.head())

shape: (5, 37)
┌────────────┬──────────┬────────┬─────────────────────┬─────────────────┬──────────┬──────────┬──────────────┬─────────────┬─────────────────────┬─────────────────────┬─────────────────────┬───────────────────┬────────────────────┬────────────────────┬───────────┬──────────┬────────────────┬────────────────────────┬─────────────────────┬─────────────────────┬──────────────────────┬───────────────────┬──────────┬────────┬────────────┬────────────┬──────┬───────────┬──────────┬──────────────────┬───────────────┬───────────────┬──────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ subject_id ┆ hadm_id  ┆ itemid ┆ charttime           ┆ serumCreatinine ┆ valueuom ┆ stay_id  ┆ itemid_urine ┆ value_urine ┆ admittime           ┆ dischtime           ┆ deathtime           ┆ admission_type    ┆ admission_location ┆ discharge_location ┆ insurance ┆ language ┆ marital_status ┆ race                   ┆ edregtime           ┆ edouttime           ┆ hospital

In [ ]:
import polars as pl
import gc
gc.collect()
# 널이 100%가 아닌 컬럼 이름 추출
columns_to_keep = []
columns_to_drop = []
total_rows = len(filtered_result)

for col in filtered_result.columns:
    null_count = filtered_result.select(pl.col(col).null_count()).item()
    if null_count != total_rows:  # 널이 100%가 아닌 경우
        columns_to_keep.append(col)
    else:
        columns_to_drop.append(col)

# filtered_result에서 해당 컬럼만 선택
filtered_result_dropped = filtered_result.select(columns_to_keep)

# 결과 확인
print("드롭 후 남은 컬럼:")
print(filtered_result_dropped.columns)

print("\n드롭된 컬럼:")
print(columns_to_drop)

AKI_data=filtered_result_dropped
del filtered_result
gc.collect()

드롭 후 남은 컬럼:
['subject_id', 'hadm_id', 'itemid', 'charttime', 'serumCreatinine', 'valueuom', 'stay_id', 'itemid_urine', 'value_urine', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime', 'hospital_expire_flag', 'admit_provider_id', 'icd_code', 'gender', 'anchor_age', 'dod', 'drug', 'starttime', 'stoptime', 'subject_id_right', 'hadm_id_right', 'stay_id_right', 'itemid_right', 'starttime_right', 'endtime', 'baseline_creatinine']

드롭된 컬럼:
[]


0

In [ ]:
import polars as pl

# surgical_itemids 정의
emergency_surgical_itemids = [
    226236, 226237, 224263, 224264, 224267, 224268, 224269, 224270,
    224272, 224273, 224274, 224275, 224276, 224277, 225752, 227550,
    227551, 228715
]
non_emergency_surgical_itemids = [
    224385, 225399, 225400, 225401, 225402, 225406, 225427, 225428,
    225429, 225430, 225432, 225433, 225434, 225436, 225437, 225439,
    225441, 225443, 225444, 225445, 225446, 225447, 225448, 225449,
    225450, 225451, 225789, 225792, 225794, 225802, 225805, 225814,
    225816, 225817, 225819, 225820, 225821, 229585
]

# surgical_itemids 리스트 정의 (emergency_surgical_itemids와 non_emergency_surgical_itemids를 합침)
surgical_itemids = emergency_surgical_itemids + non_emergency_surgical_itemids

def estimate_surgery_duration(df):
    # admittime과 dischtime을 datetime으로 변환 (벡터화)
    df = df.with_columns([
        pl.col("admittime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False).alias("admittime_dt"),
        pl.col("dischtime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False).alias("dischtime_dt"),
        pl.col("edregtime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False).alias("edregtime_dt"),
        pl.col("edouttime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False).alias("edouttime_dt"),
        # pl.col("starttime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False).alias("starttime_dt"),
        # pl.col("stoptime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False).alias("stoptime_dt")
    ])

    # 병원 체류 시간 계산 (시간 단위로 변환)
    df = df.with_columns([
        pl.when(pl.col("admittime_dt").is_not_null() & pl.col("dischtime_dt").is_not_null())
        .then((pl.col("dischtime_dt").cast(pl.Int64) - pl.col("admittime_dt").cast(pl.Int64)) / (1000 * 60 * 60))
        .alias("hospital_stay_duration"),  # null 유지

        pl.when(pl.col("edregtime_dt").is_not_null() & pl.col("edouttime_dt").is_not_null())
        .then((pl.col("edouttime_dt").cast(pl.Int64) - pl.col("edregtime_dt").cast(pl.Int64)) / (1000 * 60 * 60))
        .alias("ed_stay_duration"),  # null 유지
    ])


    return df



In [ ]:
import polars as pl
import gc

# 당뇨병 여부를 판단하는 Polars 표현식 함수
def is_diabetes_code(icd_code_column):
    return (
        icd_code_column.str.starts_with("250") |  # ICD-9
        icd_code_column.str.starts_with("E10") |  # ICD-10 Type 1
        icd_code_column.str.starts_with("E11") |  # ICD-10 Type 2
        icd_code_column.str.starts_with("E13")    # ICD-10 Other specified
    )

# AKI 정의 함수
def define_aki(creatinine_results):
    print("Defining AKI...")
    gc.collect()
    #filtered_result로 부터 기저 creatinine : baseline_creatinine
    # creatinine_results=
    SPARK_results = creatinine_results.rename({"anchor_age": "age"})

    # 성별에 따른 eGFR 계산에 사용되는 상수
    female_k, male_k = 0.7, 0.9
    female_alpha, male_alpha = -0.241, -0.302
    female_factor, male_factor = 1.012, 1.0

    # eGFR_2021 컬럼이 없는 경우 기본값으로 생성
    if 'eGFR_2021' not in SPARK_results.columns:
        SPARK_results = SPARK_results.with_columns([
            pl.lit(None).cast(pl.Float64).alias("eGFR_2021")
        ])

    SPARK_results = SPARK_results.with_columns([
        pl.when(pl.col('itemid').is_in([50912]))
        .then(
            (142 *
            (pl.when(pl.col("serumCreatinine") / pl.when(pl.col("gender") == "F").then(female_k).otherwise(male_k) < 1)
            .then(pl.col("serumCreatinine") / pl.when(pl.col("gender") == "F").then(female_k).otherwise(male_k))
            .otherwise(1) ** pl.when(pl.col("gender") == "F").then(female_alpha).otherwise(male_alpha)) *
            (pl.when(pl.col("serumCreatinine") / pl.when(pl.col("gender") == "F").then(female_k).otherwise(male_k) > 1)
            .then(pl.col("serumCreatinine") / pl.when(pl.col("gender") == "F").then(female_k).otherwise(male_k))
            .otherwise(1) ** -1.200) *
            (0.993 ** pl.col("age")) *
            pl.when(pl.col("gender") == "F").then(female_factor).otherwise(male_factor))
        )
        .otherwise(pl.col("eGFR_2021"))
        .alias("eGFR_2021")
    ])
    SPARK_results = SPARK_results.with_columns([# SPARK_score 초기화
        pl.lit(0).alias("SPARK_score")
    ])

    # 나이에 따른 점수 계산
    SPARK_results = SPARK_results.with_columns([
        (pl.col("SPARK_score") +
        pl.when(pl.col('age') < 40).then(0)
        .when((pl.col('age') >= 40) & (pl.col('age') < 60)).then(6)
        .when((pl.col('age') >= 60) & (pl.col('age') < 80)).then(9)
        .otherwise(13)  # 나이에 따른 점수
        ).alias("SPARK_score")
    ])

    # 중간 점수 출력 (나이 적용 후)
    print("SPARK_score after age adjustment:")

    # 당뇨 및 약물 조건 적용
    medications_of_interest = ["ARB", "Spironolactone", "Eplerenone"]
    SPARK_results = SPARK_results.with_columns([
        (
            pl.col("SPARK_score") +
            pl.when(is_diabetes_code(pl.col('icd_code'))).then(4).otherwise(0) +  # 당뇨 코드에 따른 점수
            pl.when(pl.col('drug').str.to_lowercase().is_in([med.lower() for med in medications_of_interest]))
            .then(7).otherwise(0)  # 약물에 따른 점수
        ).alias('SPARK_score')
    ])

    # 중간 점수 출력 (당뇨 및 약물 적용 후)
    print("SPARK_score after diabetes and drug adjustment:")

    # Hypoalbuminemia, 성별에 따른 점수, 수술 시간 가산점 적용
    SPARK_results = SPARK_results.with_columns([
        (
            pl.col("SPARK_score") +
            # Hypoalbuminemia
            pl.when(
                (pl.col('itemid').is_in([50862, 53085])) &  # 알부민 itemid
                (pl.col('serumCreatinine') < 3.5) &  # 알부민 수치가 기준 미만
                (pl.col('valueuom') == 'g/dL')
            ).then(8).otherwise(0) +

            # 성별에 따른 점수
            pl.when(pl.col('gender') == 'M').then(8).otherwise(0) +

            # Hyponatremia
            pl.when(
                (pl.col('itemid').is_in([50824, 50983, 52623])) &  # 나트륨 itemid
                (pl.col('serumCreatinine') < 135) &  # 나트륨 수치 기준
                (pl.col('valueuom') == 'mEq/L')
            ).then(3).otherwise(0)
        ).alias('SPARK_score')
    ])

    # 중간 점수 출력 (기타 조건 적용 후)
    print("SPARK_score after other conditions adjustment:")

    SPARK_results = SPARK_results.with_columns([
        (
            pl.col("SPARK_score") +
            pl.when(pl.col('eGFR_2021') >= 60).then(0)
            .when(pl.col('eGFR_2021').is_between(45, 60, closed='left')).then(8)
            .when(pl.col('eGFR_2021').is_between(30, 45, closed='left')).then(15)
            .when(pl.col('eGFR_2021').is_between(15, 30, closed='left')).then(22)
            .otherwise(0)
        ).alias("SPARK_score")
    ])
    # 최종 점수 출력
    print("Final SPARK_score:")

    # SPARK_class 할당
    SPARK_results = SPARK_results.with_columns([
        pl.when(pl.col('SPARK_score') < 20).then(pl.lit('A'))
        .when((pl.col('SPARK_score') >= 20) & (pl.col('SPARK_score') <= 39)).then(pl.lit('B'))
        .when((pl.col('SPARK_score') >= 40) & (pl.col('SPARK_score') <= 59)).then(pl.lit('C'))
        .otherwise(pl.lit('D')).alias('SPARK_class')
    ])

    return SPARK_results

gc.collect()



0

In [ ]:
# import gc

# # SPARK_results를 제외한 모든 변수 삭제
# globals_copy = globals().copy()  # globals() 복사본 생성

# # 'SPARK_results'를 제외한 모든 데이터프레임 변수 삭제
# for var_name in globals_copy:
#     if var_name != "SPARK_results" and isinstance(globals()[var_name], pl.DataFrame):
#         del globals()[var_name]

# # 가비지 컬렉터 실행
# gc.collect()

# print("SPARK_results를 제외한 모든 데이터프레임이 삭제되었습니다.")


In [ ]:
base_path = '/content/drive/MyDrive/AKI_공유/3.0'
icu_path = os.path.join(base_path, 'icu')
temp_dir = '/content/drive/MyDrive/AKI_공유/3.0/full_join'
pl.Config.set_tbl_rows(1000)# 출력할 행의 수를 설정 (예: 10)
pl.Config.set_tbl_cols(100)# 출력할 열의 수를 설정 (예: 30)
pl.Config.set_tbl_width_chars(1000)

emergency_surgical_itemids = [
    226236, 226237, 224263, 224264, 224267, 224268, 224269, 224270,
    224272, 224273, 224274, 224275, 224276, 224277, 225752, 227550,
    227551, 228715
]
non_emergency_surgical_itemids = [
    224385, 225399, 225400, 225401, 225402, 225406, 225427, 225428,
    225429, 225430, 225432, 225433, 225434, 225436, 225437, 225439,
    225441, 225443, 225444, 225445, 225446, 225447, 225448, 225449,
    225450, 225451, 225789, 225792, 225794, 225802, 225805, 225814,
    225816, 225817, 225819, 225820, 225821, 229585
]
parquet_path_procedureevents = os.path.join(temp_dir, '8-procedureevents.parquet')


In [ ]:
# emergency_surgical_itemids 및 non_emergency_surgical_itemids 정의
emergency_surgical_itemids = [
    226236, 226237, 224263, 224264, 224267, 224268, 224269, 224270,
    224272, 224273, 224274, 224275, 224276, 224277, 225752, 227550,
    227551, 228715
]
non_emergency_surgical_itemids = [
    224385, 225399, 225400, 225401, 225402, 225406, 225427, 225428,
    225429, 225430, 225432, 225433, 225434, 225436, 225437, 225439,
    225441, 225443, 225444, 225445, 225446, 225447, 225448, 225449,
    225450, 225451, 225789, 225792, 225794, 225802, 225805, 225814,
    225816, 225817, 225819, 225820, 225821, 229585
]

# 모든 itemid 리스트 생성
all_itemids = emergency_surgical_itemids + non_emergency_surgical_itemids

# procedureevents.csv 로드
procedureevents = pl.scan_csv(
    os.path.join(icu_path, 'procedureevents.csv'),
    infer_schema_length=100,  # 스키마 추론 길이 설정
    ignore_errors=True  # 오류 무시 설정
)

# itemid로 필터링하고 starttime, endtime이 있는 경우에만 남김
filtered_procedureevents = procedureevents.select([
    'subject_id', 'hadm_id', 'stay_id', 'itemid', 'starttime', 'endtime'
]).filter(
    (pl.col("itemid").is_in(all_itemids)) &
    (pl.col("starttime").is_not_null()) &
    (pl.col("endtime").is_not_null())
)

In [ ]:
print("operation_results columns:", filtered_procedureevents.schema)
gc.collect()

operation_results columns: Schema([('subject_id', Int64), ('hadm_id', Int64), ('stay_id', Int64), ('itemid', Int64), ('starttime', String), ('endtime', String)])


<ipython-input-10-7c2092ea46f8>:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  print("operation_results columns:", filtered_procedureevents.schema)


0

In [ ]:
def estimate_surgery_duration_procedure(df):
    # admittime과 dischtime을 datetime으로 변환 (벡터화)
    df = df.with_columns([
        pl.col("starttime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False).alias("starttime_dt"),
        pl.col("endtime").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False).alias("endtime_dt")
    ])

    # 필터링: starttime과 endtime이 존재하는 경우에만 계산
    df_filtered = df.filter(
        (pl.col("starttime_dt").is_not_null()) &
        (pl.col("endtime_dt").is_not_null())
    )

    # 병원 체류 시간 계산 (시간 단위로 변환)
    df_filtered = df_filtered.with_columns([
        pl.when(pl.col("itemid").is_in(surgical_itemids))
        .then((pl.col("endtime_dt").cast(pl.Int64) - pl.col("starttime_dt").cast(pl.Int64)) / (1000 * 60 * 60))
        .alias("trauma_operation_duration"),  # null 유지

        pl.when(pl.col("itemid").is_in(emergency_surgical_itemids))
        .then((pl.col("endtime_dt").cast(pl.Int64) - pl.col("starttime_dt").cast(pl.Int64)) / (1000 * 60 * 60))
        .alias("emergency_operation_duration")  # null 유지
    ])

    # 계산된 케이스 수 확인
    trauma_cases = df_filtered.filter(pl.col("trauma_operation_duration").is_not_null()).height
    emergency_cases = df_filtered.filter(pl.col("emergency_operation_duration").is_not_null()).height

    print(f"Trauma operation duration 계산된 케이스: {trauma_cases}")
    print(f"Emergency operation duration 계산된 케이스: {emergency_cases}")

    return df_filtered


In [37]:
# SPARK_results를 LazyFrame으로 변환
SPARK_results_lazy = AKI_data.lazy()

# operation_results가 이미 LazyFrame인 경우를 고려하여 LazyFrame으로 변환
if isinstance(filtered_procedureevents, pl.LazyFrame):
    operation_results_lazy = filtered_procedureevents
else:
    operation_results_lazy = filtered_procedureevents.lazy()

# operation_results의 열 목록 확인 (Schema.names() 메서드 호출)
print("operation_results columns:", operation_results_lazy.collect_schema().names())

# '_right'로 끝나는 컬럼을 필터링하여 제거
SPARK_results_lazy_cleaned = SPARK_results_lazy.drop(
    [col for col in SPARK_results_lazy.collect_schema().names() if col.endswith('_right')]
)

# 중복된 행 제거 (모든 열 기준으로 중복된 경우만 하나만 남기기)
SPARK_results_lazy_cleaned = SPARK_results_lazy_cleaned.unique()

# operation_results에서 필요한 컬럼 선택
operation_results_lazy_filtered = operation_results_lazy.select([
    'subject_id', 'hadm_id', 'stay_id', 'itemid', 'starttime', 'endtime'
])

# itemid로 필터링하고 starttime, endtime이 있는 경우에만 남김
filtered_procedureevents = operation_results_lazy_filtered.filter(
    (pl.col("itemid").is_in(all_itemids)) &
    (pl.col("starttime").is_not_null()) &
    (pl.col("endtime").is_not_null())
)

# 필터링된 결과의 열 수 카운트
count_procedureevents = filtered_procedureevents.collect().height

# 결과 출력
print(f"--------------------------------Starttime과 Endtime이 있는 필터링된 열의 수: {count_procedureevents}")
operation_results = estimate_surgery_duration_procedure(filtered_procedureevents.collect())
print(operation_results.collect_schema().names())
print("after")
# LazyFrame을 DataFrame으로 변환
operation_results_df = operation_results_lazy.collect()
operation_results_final = estimate_surgery_duration_procedure(operation_results_df)

# SPARK_score 초기화
operation_results_final = operation_results_final.with_columns([
    pl.lit(0).alias("SPARK_score")
])

# SPARK_score 업데이트: 수술 시간과 응급 수술 가산
operation_results_final = operation_results_final.with_columns([
    (
        # 수술 시간 가산 (hospital_stay_duration * 5) null 값은 0으로 채움
        (pl.col('trauma_operation_duration') * 5).fill_null(0) +

        # 응급 수술 가산 (ed_stay_duration이 True일 때 7, 아니면 0)
        pl.when(pl.col('emergency_operation_duration') == True).then(7).otherwise(0)
    ).alias('SPARK_score')
])



print(operation_results_final.columns)
# DataFrame을 CSV 파일로 저장
operation_results_final.write_csv("/content/drive/MyDrive/AKI_공유/3.0/full_join/operation_results.csv")
SPARK_results_lazy_cleaned = SPARK_results_lazy.unique()
SPARK_results_lazy_cleaned=define_aki(SPARK_results_lazy_cleaned)
SPARK_results_lazy_cleaned.collect().write_csv("/content/drive/MyDrive/AKI_공유/3.0/full_join/total_data_before_operation_join-1.csv")


operation_results columns: ['subject_id', 'hadm_id', 'stay_id', 'itemid', 'starttime', 'endtime']
--------------------------------Starttime과 Endtime이 있는 필터링된 열의 수: 481342
Trauma operation duration 계산된 케이스: 16769
Emergency operation duration 계산된 케이스: 0
['subject_id', 'hadm_id', 'stay_id', 'itemid', 'starttime', 'endtime', 'starttime_dt', 'endtime_dt', 'trauma_operation_duration', 'emergency_operation_duration']
after
Trauma operation duration 계산된 케이스: 16769
Emergency operation duration 계산된 케이스: 0
['subject_id', 'hadm_id', 'stay_id', 'itemid', 'starttime', 'endtime', 'starttime_dt', 'endtime_dt', 'trauma_operation_duration', 'emergency_operation_duration', 'SPARK_score']
Defining AKI...
SPARK_score after age adjustment:
SPARK_score after diabetes and drug adjustment:
SPARK_score after other conditions adjustment:
Final SPARK_score:


<ipython-input-6-599f2f3ba2d1>:27: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if 'eGFR_2021' not in SPARK_results.columns:


MIMIC-IV 데이터셋에서 DIABETES MELLITUS와 RAAS blockade use에 대한 정보는 주로 다음 테이블에서 찾을 수 있습니다:
DIABETES MELLITUS:
diagnoses_icd 테이블: ICD-9 또는 ICD-10 코드를 통해 당뇨병 진단을 확인할 수 있습니다.
prescriptions 테이블: 당뇨병 치료제 처방을 통해 간접적으로 당뇨병 여부를 확인할 수 있습니다.
RAAS blockade use:
prescriptions 테이블: ACE 억제제, ARB 등 RAAS 차단제 처방 정보를 확인할 수 있습니다.
inputevents 테이블: ICU에서 투여된 RAAS 차단제 정보를 확인할 수 있습니다.
이 정보는 제공된 검색 결과에 직접적으로 언급되어 있지 않지만, MIMIC-IV 데이터베이스의 구조와 일반적인 의료 데이터 구성을 고려할 때 이러한 테이블들에서 관련 정보를 찾을 수 있을 것입니다.
예를 들어, 검색 결과 에서는 MIMIC-IV 데이터베이스를 사용하여 당뇨병 환자의 관상동맥 우회술 후 혈당 변화와 섬망 발생 간의 관계를 연구했습니다. 이 연구에서 당뇨병 환자를 식별하기 위해 diagnoses_icd 테이블을 사용했을 가능성이 높습니다.
RAAS blockade use에 대한 정보는 제공된 검색 결과에 직접적으로 언급되어 있지 않지만, 일반적으로 prescriptions 테이블에서 이러한 약물 사용 정보를 찾을 수 있습니다.
더 정확한 정보를 위해서는 MIMIC-IV 데이터베이스의 공식 문서를 참조하거나, 데이터베이스를 직접 탐색해보는 것이 도움이 될 수 있습니다

MIMIC-IV 데이터셋에서 Hypoalbuminemia, Anemia, Hyponatremia 관련 정보는 주로 laboratory events 테이블에서 찾을 수 있습니다. 제공된 검색 결과를 바탕으로 다음과 같이 정리할 수 있습니다:
Hypoalbuminemia:
테이블: labevents
관련 컬럼: itemid (알부민 검사 코드), valuenum (알부민 수치)
Anemia:
테이블: labevents
관련 컬럼: itemid (헤모글로빈 검사 코드), valuenum (헤모글로빈 수치)
Hyponatremia:
테이블: labevents
관련 컬럼: itemid (혈청 나트륨 검사 코드), valuenum (혈청 나트륨 수치)
특히, 검색 결과 에서는 혈청 나트륨 수치와 사망률의 관계를 분석하기 위해 MIMIC-IV 데이터베이스를 사용했습니다. 이 연구에서는 labevents 테이블에서 혈청 나트륨 수치를 추출했을 것으로 추정됩니다.
추가적인 정보:
itemid는 각 검사항목을 식별하는 고유 코드입니다. 예를 들어, 나트륨 검사의 itemid는 특정 번호로 지정되어 있습니다.
valuenum은 실제 검사 결과 값을 나타냅니다.
검사 결과의 단위는 valueuom 컬럼에서 확인할 수 있습니다.
이러한 정보를 얻기 위해서는 MIMIC-IV 데이터베이스의 데이터 사전(data dictionary)을 참조하여 정확한 itemid와 관련 정보를 확인해야 합니다. 또한, 환자의 기본 정보와 연결하기 위해 subject_id나 hadm_id 등의 식별자를 사용해야 할 수 있습니다.


제시된 약물들(ACEI, ARB, Spironolactone, Eplerenone)은 직접적인 당뇨병 치료제는 아니지만, 당뇨병 환자들에게 자주 처방되는 약물들입니다. 이 약물들의 주요 용도와 당뇨병과의 관련성을 설명드리겠습니다:
ACEI (Angiotensin-Converting Enzyme Inhibitors)와 ARB (Angiotensin Receptor Blockers):
주로 고혈압 치료에 사용됩니다.
당뇨병 환자에서 흔히 동반되는 고혈압 치료와 신장 보호 목적으로 사용됩니다.
당뇨병성 신증의 진행을 늦추는 데 효과적입니다.
Spironolactone과 Eplerenone:
이들은 mineralocorticoid receptor antagonists (MRAs)로 알려진 약물 그룹에 속합니다.
주로 고혈압, 심부전, 부종 등의 치료에 사용됩니다.
당뇨병 환자에서 혈압 조절과 신장 보호 효과가 있어 사용됩니다.
특히 당뇨병성 신증 환자에서 단백뇨를 감소시키는 효과가 있습니다.
검색 결과에서 볼 수 있듯이, 이 약물들은 당뇨병 환자들의 합병증 관리, 특히 신장 보호와 혈압 조절에 중요한 역할을 합니다. 예를 들어, 결과 에서는 spironolactone이 제2형 당뇨병 환자의 치료 저항성 고혈압에 효과가 있음을 보여줍니다.
따라서, 이 약물들은 직접적인 혈당 조절 약물은 아니지만, 당뇨병 환자의 전반적인 건강 관리, 특히 심혈관 및 신장 합병증 예방과 치료에 중요한 역할을 하는 약물들입니다.



    age기준  risk_factors
    1. <40 0
    2. >=40 and <60 6
    3. >=60 and <80 9
    4. >= 80        13

    CKD-EPI 2021 공식을 수학적 표현으로 나타내면 다음과 같습니다:
    eGFR = 141 × min(SCr/κ, 1)α × max(SCr/κ, 1)-1.209 × 0.993Age × 1.018 [if female]
    여기서:
    eGFR: estimated Glomerular Filtration Rate (mL/min/1.73 m²)
    SCr: 혈청 크레아티닌 농도 (mg/dL)
    κ: 0.7 (여성) 또는 0.9 (남성)
    α: -0.329 (여성) 또는 -0.411 (남성)
    Age: 나이 (년)
    이 공식을 단계별로 설명하면:
    SCr/κ 비율 계산:
    여성: SCr/0.7
    남성: SCr/0.9
    min(SCr/κ, 1)α 계산:
    여성: min(SCr/0.7, 1)-0.329
    남성: min(SCr/0.9, 1)-0.411
    max(SCr/κ, 1)-1.209 계산:
    여성: max(SCr/0.7, 1)-1.209
    남성: max(SCr/0.9, 1)-1.209
    0.993Age 계산
    성별 계수 적용:
    여성: × 1.018
    남성: × 1.0 (변화 없음)
    모든 요소를 곱하여 최종 eGFR 계산:
    eGFR = 141 × (2단계 결과) × (3단계 결과) × (4단계 결과) × (5단계 결과)
    이 공식은 이전 버전과 달리 인종 요소를 포함하지 않으며, 성별과 나이만을 고려하여 eGFR을 추정합니다.
    이는 인종에 따른 차별을 줄이고 더 공정한 신장 기능 평가를 제공하기 위한 변경사항입니다.

    eGRF (mL/min/1.73 m^2)
    >= 60 0
    >= 43 and < 60 8
    >= 30 and < 45 15
    >= 15 and < 30 22
    Dipstick albuminuria (urine albumin >= 1 + ) 6

    SPARK_score 값에 따라서 A B C D 클래스를 매길수 있어
    SPARK_score< 20 A
    SPARK_score 20~39 B
    SPARK_score 40~59 C
    SPARK_score >= 60 D

In [ ]:
# import polars as pl
# import re
# import os

# # 당뇨병 여부를 판단하는 Polars 표현식 함수
# def is_diabetes_code(icd_code_column):
#     return (
#         icd_code_column.str.starts_with("250") |  # ICD-9
#         icd_code_column.str.starts_with("E10") |  # ICD-10 Type 1
#         icd_code_column.str.starts_with("E11") |  # ICD-10 Type 2
#         icd_code_column.str.starts_with("E13")    # ICD-10 Other specified
#     )

# # AKI 정의 함수
# def define_aki(creatinine_results):
#     print("Defining AKI...")
#     gc.collect()
#     SPARK_results = creatinine_results.rename({"anchor_age": "age"})

#     female_k, male_k = 0.7, 0.9
#     female_alpha, male_alpha = -0.241, -0.302
#     female_factor, male_factor = 1.012, 1.0

#     SPARK_results = SPARK_results.with_columns([
#         (142 *
#         (pl.when(pl.col("serumCreatinine") / pl.when(pl.col("gender") == "F").then(female_k).otherwise(male_k) < 1)
#         .then(pl.col("serumCreatinine") / pl.when(pl.col("gender") == "F").then(female_k).otherwise(male_k))
#         .otherwise(1) ** pl.when(pl.col("gender") == "F").then(female_alpha).otherwise(male_alpha)) *
#         (pl.when(pl.col("serumCreatinine") / pl.when(pl.col("gender") == "F").then(female_k).otherwise(male_k) > 1)
#         .then(pl.col("serumCreatinine") / pl.when(pl.col("gender") == "F").then(female_k).otherwise(male_k))
#         .otherwise(1) ** -1.200) *
#         (0.993 ** pl.col("age")) *
#         pl.when(pl.col("gender") == "F").then(female_factor).otherwise(male_factor)
#         ).alias("eGFR_2021")
#     ])
#     SPARK_results = SPARK_results.with_columns([
#         pl.lit(0).alias("SPARK_score")  # 초기 SPARK_score는 0
#     ])
#     # 점수 계산
#     SPARK_results = SPARK_results.with_columns([
#         (pl.col("SPARK_score") +
#         pl.when(pl.col('age') < 40).then(0)
#         .when((pl.col('age') >= 40) & (pl.col('age') < 60)).then(6)
#         .when((pl.col('age') >= 60) & (pl.col('age') < 80)).then(9)
#         .otherwise(13)  # 나이에 따른 점수
#         ).alias("SPARK_score")
#     ])
#     medications_of_interest = ["ARB", "Spironolactone", "Eplerenone"]
#     # SPARK_score 계산 (각 조건의 점수를 합산)
#     SPARK_results = SPARK_results.with_columns([
#         (
#             pl.col("SPARK_score") +
#             # pl.when(pl.col('gender') == 'M').then(8).otherwise(0) +  # 성별에 따른 점수
#             pl.when(is_diabetes_code(pl.col('icd_code'))).then(4).otherwise(0) +  # 당뇨 코드에 따른 점수
#             pl.when(pl.col('drug').str.to_lowercase().is_in([med.lower() for med in medications_of_interest]))
#             .then(7).otherwise(0)  # 약물에 따른 점수
#         ).alias('SPARK_score')
#     ])
#     SPARK_results=estimate_surgery_duration(SPARK_results)
#     gc.collect()

#     # SPARK_score 누적 방식 수정
#     SPARK_results = SPARK_results.with_columns(
#         (
#             pl.col("SPARK_score") +  # 기존 SPARK_score에 계속해서 더함
#             # Hypoalbuminemia
#             pl.when(
#                 (pl.col('itemid').is_in([50862, 53085])) &
#                 (pl.col('serumCreatinine') < 3.5) &
#                 (pl.col('valueuom') == 'g/dL')
#             ).then(8).otherwise(0) +

#             # 성별에 따른 점수
#             pl.when(
#                 pl.col('gender') == 'M'
#             ).then(8).otherwise(0) +

#             # Hyponatremia(<=135 mEq/L) 혈액 내 나트륨 농도가 비정상적으로 낮은 상태
#             pl.when(
#                 (pl.col('itemid').is_in([50824, 50983, 52623])) &
#                 (pl.col('serumCreatinine') < 135) &
#                 (pl.col('valueuom') == 'mEq/L')
#             ).then(3).otherwise(0) +

#             # 수술 시간 x5 가산
#             (pl.col('hospital_stay_duration') * 5).fill_null(0) +

#             # 응급 수술 가산
#             pl.when(pl.col('ed_stay_duration') == True).then(7).otherwise(0)
#         ).alias('SPARK_score')
#     )

#     SPARK_results = SPARK_results.with_columns([
#         # 기존의 SPARK 점수 계산
#         (pl.col('hospital_stay_duration') * 5).fill_null(0) +
#         # 여기에 새로운 eGFR 및 알부민뇨 점수 추가
#         pl.when(pl.col('eGFR_2021') >= 60).then(0)
#         .when(pl.col('eGFR_2021').is_between(45, 60, closed='left')).then(8)
#         .when(pl.col('eGFR_2021').is_between(30, 45, closed='left')).then(15)
#         .when(pl.col('eGFR_2021').is_between(15, 30, closed='left')).then(22)
#         .otherwise(0)
#         # pl.when(pl.col('dipstick_albuminuria') >= 1).then(6)
#         # .otherwise(0)
#         .alias('SPARK_score')
#     ])



#     SPARK_results = SPARK_results.with_columns([
#         pl.lit("A").alias("SPARK_class")  # 초기 SPARK_class는 문자 "A"
#     ])

#     # SPARK_class 할당
#     SPARK_results = SPARK_results.with_columns([
#         pl.when(pl.col('SPARK_score') < 20).then(pl.lit('A'))
#         .when((pl.col('SPARK_score') >= 20) & (pl.col('SPARK_score') <= 39)).then(pl.lit('B'))
#         .when((pl.col('SPARK_score') >= 40) & (pl.col('SPARK_score') <= 59)).then(pl.lit('C'))
#         .otherwise(pl.lit('D')).alias('SPARK_class')
#     ])

#     return SPARK_results

# gc.collect()
# SPARK_results = define_aki(filtered_result_dropped)
# print(SPARK_results.select('SPARK_class').to_series().value_counts())


MIMIC-IV 데이터셋에서 약물 정보, 특히 "ACEI", "ARB", "Spironolactone", "Eplerenone" 같은 약물에 대한 정보는 주로 'prescriptions' 테이블에서 찾을 수 있습니다. 이 테이블은 환자에게 처방된 약물 정보를 포함하고 있습니다.
'prescriptions' 테이블의 주요 컬럼들은 다음과 같습니다:
subject_id: 환자 고유 식별자
hadm_id: 입원 고유 식별자
pharmacy_id: 약국 고유 식별자
starttime: 약물 투여 시작 시간
stoptime: 약물 투여 종료 시간
drug_type: 약물 유형
drug: 약물 이름
gsn: Generic Sequence Number
ndc: National Drug Code
prod_strength: 약물 강도
dose_val_rx: 처방된 용량 값
dose_unit_rx: 처방된 용량 단위
form_val_disp: 조제된 형태 값
form_unit_disp: 조제된 형태 단위
약물 이름을 찾으려면 주로 'drug' 컬럼을 사용합니다. 그러나 약물 이름이 다양한 형태로 기록될 수 있으므로 (예: 상품명, 일반명 등), 검색 시 주의가 필요합니다.

import polars as pl

medications_of_interest = ["ACEI", "ARB", "Spironolactone", "Eplerenone"]

filtered_prescriptions = prescriptions.filter(
    pl.col("drug").str.to_lowercase().is_in([med.lower() for med in medications_of_interest])
)

print(filtered_prescriptions)

In [ ]:
import polars as pl
import re
import os
base_path = '/content/drive/MyDrive/AKI_공유/3.0'
hosp_path = os.path.join(base_path, 'hosp')
icu_path = os.path.join(base_path, 'icu')


d_items = pl.read_csv(
    os.path.join(icu_path,"d_items.csv.gz"),
    dtypes={'highnormalvalue': pl.Float64}
)

# 관련 항목 찾기
trauma_emergency_items = d_items.filter(
    pl.col("label").str.to_lowercase().str.contains("surgery|procedure")
)

# 결과 출력
print(trauma_emergency_items.select(["itemid", "label", "category", "abbreviation"]))

procedureevents = pl.read_csv(
    os.path.join(icu_path, "procedureevents.csv"),
    schema_overrides={"originalamount": pl.Float64}
)

print(procedureevents.columns)
procedure_categories = [
    "4-Procedures",  # 일반적인 의료 시술 카테고리
    "1-Intubation/Extubation",  # 기관 삽관 및 제거 관련 시술
    "2-Ventilation",  # 인공 호흡 관련 시술
    "3-Significant Events",  # 중요한 의료 이벤트 관련 시술
    "PICC Line Insertion",  # 말초 삽입 중심 정맥 카테터 삽입
    "CVL Insertion",  # 중심 정맥관 삽입
    "Arterial Line Insertion",  # 동맥관 삽입
    "PA Line Insertion",  # 폐동맥 카테터 삽입
    "Bronchoscopy",  # 기관지 내시경 검사
    "Lumbar Puncture",  # 요추 천자
    "Thoracentesis",  # 흉강 천자
    "Paracentesis"  # 복강 천자
]

procedure_items = d_items.filter(
    pl.col("category").is_in(procedure_categories)
)

print(procedure_items.select(["itemid", "label", "category"]))

print(procedure_items)

shape: (39, 4)
┌────────┬─────────────────────────────────┬─────────────────────────┬─────────────────────────────────┐
│ itemid ┆ label                           ┆ category                ┆ abbreviation                    │
│ ---    ┆ ---                             ┆ ---                     ┆ ---                             │
│ i64    ┆ str                             ┆ str                     ┆ str                             │
╞════════╪═════════════════════════════════╪═════════════════════════╪═════════════════════════════════╡
│ 225283 ┆ Hand Cleansing Prior to Proced… ┆ PICC Line Insertion     ┆ Hand Cleansing Prior to Proced… │
│ 225486 ┆ Hand cleansing prior to proced… ┆ Thoracentesis           ┆ Hand cleansing prior to proced… │
│ 225503 ┆ Hand cleansing prior to proced… ┆ Paracentesis            ┆ Hand cleansing prior to proced… │
│ 225536 ┆ Hand cleansing prior to proced… ┆ Bronchoscopy            ┆ Hand cleansing prior to proced… │
│ 225567 ┆ Hand cleansing prior to proce

<ipython-input-20-5de8c19140b8>:9: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  d_items = pl.read_csv(


['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'starttime', 'endtime', 'storetime', 'itemid', 'value', 'valueuom', 'location', 'locationcategory', 'orderid', 'linkorderid', 'ordercategoryname', 'ordercategorydescription', 'patientweight', 'isopenbag', 'continueinnextdept', 'statusdescription', 'originalamount', 'originalrate']
shape: (430, 3)
┌────────┬─────────────────────────────────┬─────────────────────────┐
│ itemid ┆ label                           ┆ category                │
│ ---    ┆ ---                             ┆ ---                     │
│ i64    ┆ str                             ┆ str                     │
╞════════╪═════════════════════════════════╪═════════════════════════╡
│ 221223 ┆ EEG                             ┆ 4-Procedures            │
│ 224336 ┆ Type of catheter (CVL)          ┆ CVL Insertion           │
│ 224337 ┆ Insertion site (CVL)            ┆ CVL Insertion           │
│ 224339 ┆ Guidewire exchange (CVL)        ┆ CVL Insertion           │
│ 224340 ┆

In [ ]:
print(procedureevents.columns)

['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'starttime', 'endtime', 'storetime', 'itemid', 'value', 'valueuom', 'location', 'locationcategory', 'orderid', 'linkorderid', 'ordercategoryname', 'ordercategorydescription', 'patientweight', 'isopenbag', 'continueinnextdept', 'statusdescription', 'originalamount', 'originalrate']


In [ ]:
base_path = '/content/drive/MyDrive/AKI_공유/3.0'
hosp_path = os.path.join(base_path, 'hosp')
icu_path = os.path.join(base_path, 'icu')


d_items = pl.read_csv(
    os.path.join(icu_path,"d_items.csv.gz"),
    dtypes={'highnormalvalue': pl.Float64}
)

# 관련 항목 찾기
trauma_emergency_items = d_items.filter(
    pl.col("label").str.to_lowercase().str.contains("surgery|procedure")
)

# 결과 출력
print(trauma_emergency_items.select(["itemid", "label", "category", "abbreviation"]))

procedureevents = pl.read_csv(
    os.path.join(icu_path, "procedureevents.csv"),
    schema_overrides={"originalamount": pl.Float64}
)
procedure_categories = [
    "4-Procedures",  # 일반적인 의료 시술 카테고리
    "1-Intubation/Extubation",  # 기관 삽관 및 제거 관련 시술
    "2-Ventilation",  # 인공 호흡 관련 시술
    "3-Significant Events",  # 중요한 의료 이벤트 관련 시술
    "PICC Line Insertion",  # 말초 삽입 중심 정맥 카테터 삽입
    "CVL Insertion",  # 중심 정맥관 삽입
    "Arterial Line Insertion",  # 동맥관 삽입
    "PA Line Insertion",  # 폐동맥 카테터 삽입
    "Bronchoscopy",  # 기관지 내시경 검사
    "Lumbar Puncture",  # 요추 천자
    "Thoracentesis",  # 흉강 천자
    "Paracentesis"  # 복강 천자
]

procedure_items = d_items.filter(
    pl.col("category").is_in(procedure_categories)
)

print(procedure_items.select(["itemid", "label", "category"]))

print(procedure_items)

# 찾아봐야 할 itemid 리스트
itemid_list = [
    224444,  # Surgical Procedure at Bedside (침상에서의 외과적 시술)
    225804,  # Surgical Procedure Outside OR (수술실 외부에서의 외과적 시술)
    225809,  # Surgical Procedure in OR (수술실에서의 외과적 시술)
    224869,  # Invasive Procedure (침습적 시술)
    225400,  # Insertion Date of Invasive Device (침습적 장치 삽입 날짜)
    224701,  # Removal Date of Invasive Device (침습적 장치 제거 날짜)
    224138,  # Procedure Start Time (시술 시작 시간)
    224139,  # Procedure Stop Time (시술 종료 시간)
    224385,  # Procedure Type (시술 유형)
    224737,  # Procedure Location (시술 위치)
    224736,  # Procedure Physician (시술 의사)
    224147,  # Procedure Note (시술 노트)
    225154,  # Procedure Complications (시술 합병증)
    228108,  # Emergency Postoperative Diagnosis (응급 수술 후 진단)
    228109,  # Emergency Postoperative Procedure (응급 수술 후 시술)
    228110,  # Emergency Preoperative Diagnosis (응급 수술 전 진단)
    228111,  # Emergency Preoperative Procedure (응급 수술 전 시술)
    228112,  # Elective Postoperative Diagnosis (선택적 수술 후 진단)
    228113,  # Elective Postoperative Procedure (선택적 수술 후 시술)
    228114,  # Elective Preoperative Diagnosis (선택적 수술 전 진단)
    228115,  # Elective Preoperative Procedure (선택적 수술 전 시술)
    224252,  # Trauma Admission (외상 입원)
    224253,  # Trauma Service (외상 서비스)
    224254,  # Trauma Type (외상 유형)
    224255,  # Trauma Team Activated (외상 팀 활성화)
    224382,  # Code Status (코드 상태 - 응급 상황 대응 계획)
    229585,  # Surgical Procedure at Bedside (침상)
    229585,  # Surgical Procedure at Bedside (침상에서의 외과적 시술)
    224263,  # Extubation (발관)
    224385,  # Intubation (삽관)
    224834,  # Tracheostomy Tube Insertion (기관절개관 삽입)
    225752,  # Tracheostomy Tube Removal (기관절개관 제거)
    224270,  # Chest Tube Insertion (흉관 삽입)
    224271,  # Chest Tube Removal (흉관 제거)
    224272,  # Central Line Insertion (중심정맥관 삽입)
    224273,  # Central Line Removal (중심정맥관 제거)
    224264,  # Dialysis Catheter Insertion (투석 카테터 삽입)
    224265,  # Dialysis Catheter Removal (투석 카테터 제거)
    224276,  # Arterial Line Insertion (동맥관 삽입)
    224277,  # Arterial Line Removal (동맥관 제거)
    224278,  # Bronchoscopy (기관지 내시경)
    224279,  # Thoracentesis (흉강천자)
    224280,  # Lumbar Puncture (요추천자)
    224281,  # Paracentesis (복강천자)
    226237,  # Code Blue (심폐소생술 필요 상황)
    226238,  # Code Status Changed (코드 상태 변경)
]

print("찾아봐야 할 itemid 리스트:")
print(itemid_list)

# 수술 관련 itemid 리스트
surgical_itemids = [
    224444, 225804, 225809, 224869, 225400, 224701, 224138, 224139, 224385, 224737,
    224736, 224147, 225154, 228108, 228109, 228110, 228111, 228112, 228113, 228114,
    228115, 224252, 224253, 224254, 224255, 224382, 229585
]

# 응급 수술 관련 itemid 리스트
emergency_surgical_itemids = [228108, 228109, 228110, 228111]

relevant_procedures = procedureevents.filter(
    pl.col("itemid").is_in(itemid_list)
)

print(relevant_procedures)

shape: (39, 4)
┌────────┬─────────────────────────────────┬─────────────────────────┬─────────────────────────────────┐
│ itemid ┆ label                           ┆ category                ┆ abbreviation                    │
│ ---    ┆ ---                             ┆ ---                     ┆ ---                             │
│ i64    ┆ str                             ┆ str                     ┆ str                             │
╞════════╪═════════════════════════════════╪═════════════════════════╪═════════════════════════════════╡
│ 225283 ┆ Hand Cleansing Prior to Proced… ┆ PICC Line Insertion     ┆ Hand Cleansing Prior to Proced… │
│ 225486 ┆ Hand cleansing prior to proced… ┆ Thoracentesis           ┆ Hand cleansing prior to proced… │
│ 225503 ┆ Hand cleansing prior to proced… ┆ Paracentesis            ┆ Hand cleansing prior to proced… │
│ 225536 ┆ Hand cleansing prior to proced… ┆ Bronchoscopy            ┆ Hand cleansing prior to proced… │
│ 225567 ┆ Hand cleansing prior to proce

<ipython-input-22-e9d263591723>:6: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  d_items = pl.read_csv(


shape: (182_995, 22)
┌────────────┬──────────┬──────────┬──────────────┬─────────────────────┬─────────────────────┬─────────────────────────┬────────┬───────────┬──────────┬──────────────────────────┬───────────────────┬─────────┬─────────────┬───────────────────────┬──────────────────────────┬───────────────┬───────────┬────────────────────┬───────────────────┬────────────────┬──────────────┐
│ subject_id ┆ hadm_id  ┆ stay_id  ┆ caregiver_id ┆ starttime           ┆ endtime             ┆ storetime               ┆ itemid ┆ value     ┆ valueuom ┆ location                 ┆ locationcategory  ┆ orderid ┆ linkorderid ┆ ordercategoryname     ┆ ordercategorydescription ┆ patientweight ┆ isopenbag ┆ continueinnextdept ┆ statusdescription ┆ originalamount ┆ originalrate │
│ ---        ┆ ---      ┆ ---      ┆ ---          ┆ ---                 ┆ ---                 ┆ ---                     ┆ ---    ┆ ---       ┆ ---      ┆ ---                      ┆ ---               ┆ ---     ┆ ---         ┆ 